In [1]:
import ml_collections
from ml_collections.config_dict import ConfigDict
import torch
import sys
from run import main
import yaml
from IPython.display import Image
import pandas as pd

In [2]:
config_dir = 'configs/config.yaml'

with open(config_dir) as file:
    config = ml_collections.ConfigDict(yaml.safe_load(file))

# General config

Choose GPU and fix the random seed

In [3]:
config.update({"device": "cuda"}, allow_val_change=True)
config.update({"gpu_id": "0"}, allow_val_change=True)
config.update({"seed": 0}, allow_val_change=True)

# Load data

We use SPX stock data with rolling window length of 20

In [4]:
sample_data = torch.load('data/stock/processed_data_20/train_X.pt')
config.update({"dataset": "custom_dataset"}, allow_val_change=True)

# Model architecture setup

We train a TimeGAN model

In [5]:
config.Model.TimeGAN.update({"generator": "LSTM"}, allow_val_change=True)
config.Model.TimeGAN.update({"discriminator": "LSTM"}, allow_val_change=True)
config.Model.TimeGAN.update({"G_hidden_dim": 64}, allow_val_change=True)
config.Model.TimeGAN.update({"G_num_layers": 2}, allow_val_change=True)
config.Model.TimeGAN.update({"D_hidden_dim": 32}, allow_val_change=True)
config.Model.TimeGAN.update({"D_num_layers": 3}, allow_val_change=True)

# Training config setup

In [6]:
# Set to number of trianing iterations to 5
config.Model.TimeGAN.update({"steps": 5}, allow_val_change=True)
config.Model.TimeGAN.update({"lr_G": 0.002}, allow_val_change=True)
config.Model.TimeGAN.update({"lr_D": 0.003}, allow_val_change=True)

### Train the model

In [7]:
config.update({"train": True}, allow_val_change=True)
config.update({"evaluate": False}, allow_val_change=True)

In [8]:
main(config = config, algo = "TimeGAN", dataset = sample_data)

data shape: torch.Size([64, 20, 2])
TimeGAN
custom_dataset_TimeGAN
GPU's available: 1
WARNING! The model exists in directory and will be overwritten
{'D_hidden_dim': 32, 'D_num_layers': 3, 'D_steps_per_G_step': 3, 'Evaluation': {'TestMetrics': {'acf_loss': {'stationary': True}, 'cov_loss': 'None', 'cross_corr': 'None', 'discriminative_score': {'batch_size': 128, 'epochs': 10, 'hidden_size': 32, 'num_layers': 2}, 'hist_loss': {'n_bins': 50}, 'icd': 'None', 'innd': 'None', 'onnd': 'None', 'permutation_test': {'n_permutation': 5}, 'predictive_score': {'batch_size': 128, 'epochs': 10, 'hidden_size': 32, 'num_layers': 2}, 'sig_mmd': {'depth': 5}, 'sigw1_loss': {'depth': 2, 'normalise': False}}, 'batch_size': 256, 'metrics_enabled': ['discriminative_score', 'predictive_score', 'sigw1', 'hist_loss', 'acf_loss', 'cov_loss', 'cross_corr', 'onnd', 'innd', 'icd'], 'n_eval': 5, 'sample_size': 1000, 'test_ratio': 0.2}, 'G_hidden_dim': 64, 'G_input_dim': 5, 'G_num_layers': 2, 'WANDB': {'log': {'imag

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:05<00:00,  1.10s/it]
wandb: WARNING Symlinked 5 files into the W&B run directory, call wandb.save again to sync new files.
wandb: WARNING Symlinked 4 files into the W&B run directory, call wandb.save again to sync new files.
wandb: WARNING Symlinked 2 files into the W&B run directory, call wandb.save again to sync new files.


# Switch to evaluation

In [5]:
config.update({"train": False}, allow_val_change=True)
config.update({"evaluate": True}, allow_val_change=True)

In [6]:
print(config.Evaluation.metrics_enabled)

['discriminative_score', 'predictive_score', 'sigw1', 'hist_loss', 'acf_loss', 'cov_loss', 'cross_corr', 'onnd', 'innd', 'icd']


In [7]:
main(config = config, algo = "TimeGAN", dataset = sample_data)

data shape: torch.Size([64, 20, 2])
TimeGAN
custom_dataset_TimeGAN
GPU's available: 1
---- evaluation metric = hist_loss in group = stylized_fact_scores ----


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 28.33it/s]


---- evaluation metric = acf_loss in group = stylized_fact_scores ----


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 89.59it/s]


---- evaluation metric = cov_loss in group = stylized_fact_scores ----


...

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:17<00:00,  3.40s/it]


[t-SNE] Computing 121 nearest neighbors...
[t-SNE] Indexed 1080 samples in 0.000s...


Exception ignored on calling ctypes callback function: <function _ThreadpoolInfo._find_modules_with_dl_iterate_phdr.<locals>.match_module_callback at 0x7f8f9c468c10>
Traceback (most recent call last):
  File "/home/jtao/ac3/lib/python3.9/site-packages/threadpoolctl.py", line 400, in match_module_callback
    self._make_module_from_path(filepath)
  File "/home/jtao/ac3/lib/python3.9/site-packages/threadpoolctl.py", line 515, in _make_module_from_path
    module = module_class(filepath, prefix, user_api, internal_api)
  File "/home/jtao/ac3/lib/python3.9/site-packages/threadpoolctl.py", line 606, in __init__
    self.version = self.get_version()
  File "/home/jtao/ac3/lib/python3.9/site-packages/threadpoolctl.py", line 646, in get_version
    config = get_config().split()
AttributeError: 'NoneType' object has no attribute 'split'


[t-SNE] Computed neighbors for 1080 samples in 0.462s...
[t-SNE] Computed conditional probabilities for sample 1000 / 1080
[t-SNE] Computed conditional probabilities for sample 1080 / 1080
[t-SNE] Mean sigma: 0.000158
[t-SNE] KL divergence after 250 iterations with early exaggeration: 61.059402
[t-SNE] KL divergence after 300 iterations: 1.087111


wandb: WARNING Symlinked 7 files into the W&B run directory, call wandb.save again to sync new files.
wandb: WARNING Symlinked 4 files into the W&B run directory, call wandb.save again to sync new files.
wandb: WARNING Symlinked 0 file into the W&B run directory, call wandb.save again to sync new files.


The evaluation results, together with the trained model will be saved under "numerical_results" forlde

In [8]:
res_dir = "./numerical_results/custom_dataset/algo_TimeGAN_G_LSTM_D_LSTM_includeD_False_n_lag_20_0/"

# Test metrics

The evaluation is run over 5 iterations, the mean ans standard deviation are collected and stored as "final_results.csv"

In [10]:
df = pd.read_csv(res_dir+'final_results.csv').T
df

,0
Unnamed: 0,0.000000
hist_loss_mean,0.218340
hist_loss_std,0.005722
acf_loss_mean,4.096016
acf_loss_std,0.029237
cov_loss_mean,0.232300
cov_loss_std,0.005398
cross_corr_mean,1.139979
cross_corr_std,0.007666
discriminative_score_mean,0.500000
